In [1]:
import tkinter as tk
from tkinter import messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import numpy as np
import time

# Import GA class from Python file
from integer_programming import GeneticAlgorithmWOC  

class GA_GUI_Visualizer:
    def __init__(self, master, ga_instance, c=None, A=None, b=None, x_ip=None):
        self.master = master
        self.ga = ga_instance

        # Use provided problem data or fall back to GA attributes if available
        if c is None or A is None or b is None:
            try:
                self.c = np.array(ga_instance.c, dtype=float)
                self.A = np.array(ga_instance.A, dtype=float)
                self.b = np.array(ga_instance.b, dtype=float)
            except AttributeError:
                self.c, self.A, self.b = np.array([]), np.array([]), np.array([])
        else:
            self.c = np.array(c, dtype=float)
            self.A = np.array(A, dtype=float)
            self.b = np.array(b, dtype=float)

        self.x_ip = np.array(x_ip, dtype=float) if x_ip is not None else None

        master.title("GA + LP Solver Visualizer")

        # ---------------- Input Frame ----------------
        self.frame_input = tk.Frame(master)
        self.frame_input.pack(pady=5)

        tk.Label(self.frame_input, text="Seed:").grid(row=0, column=0)
        self.seed_entry = tk.Entry(self.frame_input, width=8)
        self.seed_entry.grid(row=0, column=1)
        self.seed_entry.insert(0, "42")

        self.run_button = tk.Button(master, text="Run GA", command=self.run_ga)
        self.run_button.pack(pady=5)

        # ---------------- Plot Figure ----------------
        self.fig, self.ax = plt.subplots(2, 1, figsize=(6, 8))
        self.canvas = FigureCanvasTkAgg(self.fig, master)
        self.canvas.get_tk_widget().pack()

        # ---------------- Save Snapshot Button ----------------
        self.save_button = tk.Button(master, text="Save Snapshot", command=self.save_snapshot)
        self.save_button.pack(pady=5)

    def run_ga(self):
        try:
            seed = int(self.seed_entry.get())
        except ValueError:
            messagebox.showerror("Input Error", "Seed must be an integer!")
            return

        # Enable LP relaxation if available
        if hasattr(self.ga, 'use_lp_relaxation'):
            self.ga.use_lp_relaxation = True

        t0 = time.perf_counter()
        results = self.ga.genetic_alg_main(seed=seed)
        dt = time.perf_counter() - t0

        x_ga = np.array(results.get('optimal_solution', []), dtype=float)
        obj_ga = np.dot(self.c, x_ga) if x_ga.size > 0 else None

        if self.x_ip is not None and obj_ga is not None:
            obj_ip = np.dot(self.c, self.x_ip)
            rel_error = np.abs(obj_ga - obj_ip)/np.abs(obj_ip)
            feasible = np.all(self.A @ x_ga <= self.b + 1e-9)
        else:
            obj_ip, rel_error, feasible = None, None, None

        messagebox.showinfo(
            "Results",
            f"GA Objective: {obj_ga if obj_ga is not None else 'N/A'}\n"
            f"IP Objective: {obj_ip if obj_ip is not None else 'N/A'}\n"
            f"Relative Error: {rel_error if rel_error is not None else 'N/A'}\n"
            f"GA feasible? {feasible if feasible is not None else 'N/A'}\n"
            f"Runtime: {dt:.2f}s"
        )

        self.plot_results(results, x_ga)

    def plot_results(self, results, x_ga):
        self.ax[0].clear()
        self.ax[0].plot(results.get('history_of_best', []), label='GA Best Fitness', color='blue')
        self.ax[0].set_xlabel("Generation")
        self.ax[0].set_ylabel("Fitness")
        self.ax[0].set_title("GA Convergence")
        self.ax[0].legend()
        self.ax[0].grid(True)

        self.ax[1].clear()
        indices = np.arange(len(x_ga))
        self.ax[1].bar(indices, x_ga, label="GA Solution", alpha=0.7)
        if self.x_ip is not None:
            self.ax[1].bar(indices, self.x_ip, label="IP Solution", alpha=0.4)
        self.ax[1].set_xlabel("Decision Variable Index")
        self.ax[1].set_ylabel("Value")
        self.ax[1].set_title("GA vs IP Solution")
        self.ax[1].legend()
        self.ax[1].grid(True)

        self.canvas.draw()

    def save_snapshot(self):
        filename = f"GA_snapshot_{int(time.time())}.png"
        self.fig.savefig(filename, dpi=150)
        messagebox.showinfo("Snapshot Saved", f"Snapshot saved as {filename}")


# ---------------- Main runner ----------------
if __name__ == "__main__":
    # Example data (replace with your actual problem data)
    c = [1, 2, 3, 4]
    A = [[1, 2, 0, 0],
         [0, 0, 1, 1]]
    b = [10, 5]
    x_ip = [2, 1, 1, 0]

    # Initialize GA instance
    ga_instance = GeneticAlgorithmWOC()  # Adjust if constructor needs parameters

    # Launch GUI
    root = tk.Tk()
    app = GA_GUI_Visualizer(root, ga_instance, c, A, b, x_ip)
    root.mainloop()

ModuleNotFoundError: No module named 'pulp'